In [17]:
# ===========================
# 02_Recoloring.ipynb
# ===========================

import os, glob
import numpy as np
import cv2
from skimage import color

# ---------- Paths ----------
PROJECT_DIR  = "/content/drive/MyDrive/Colab Notebooks/final_project_computer_vision" #Change this to the root of where the poject is
DATA_DIR     = f"{PROJECT_DIR}/data"
IMAGES_DIR   = f"{DATA_DIR}/images"
OUT_DIR      = f"{PROJECT_DIR}/outputs"
RECOLOR_DIR  = f"{PROJECT_DIR}/outputs/recolor"
os.makedirs(RECOLOR_DIR, exist_ok=True)

# ---------- Utilities ----------
def hex_to_rgb(hex_color: str) -> np.ndarray:
    hex_color = hex_color.strip().replace("#","")
    return np.array([int(hex_color[0:2],16), int(hex_color[2:4],16), int(hex_color[4:6],16)], dtype=np.uint8)

def ensure_mask_size(mask_u8: np.ndarray, H: int, W: int) -> np.ndarray:
    if mask_u8.ndim == 3:
        mask_u8 = cv2.cvtColor(mask_u8, cv2.COLOR_BGR2GRAY)
    if mask_u8.dtype != np.uint8:
        mask_u8 = mask_u8.astype(np.uint8)
    if mask_u8.max() in (0, 1):
        mask_u8 = (mask_u8 * 255).astype(np.uint8)
    if mask_u8.shape[:2] != (H, W):
        mask_u8 = cv2.resize(mask_u8, (W, H), cv2.INTER_NEAREST)
    return mask_u8

def load_mask(path: str, H: int, W: int) -> np.ndarray:
    m = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if m is None:
        raise FileNotFoundError(f"Missing mask: {path}")
    return ensure_mask_size(m, H, W)

def find_source_image(stem: str) -> str:
    """Find the original image for a given stem inside IMAGES_DIR."""
    for ext in (".jpg", ".jpeg", ".png"):
        cand = os.path.join(IMAGES_DIR, stem + ext)
        if os.path.isfile(cand):
            return cand
    raise FileNotFoundError(f"Image not found for stem='{stem}' in {IMAGES_DIR}")

# ---------- Recolor functions (hard masks) ----------
def recolor_hsv_hard(img_rgb: np.ndarray, mask_u8: np.ndarray, target_rgb: np.ndarray, strength: float = 0.85) -> np.ndarray:
    a = (mask_u8 > 0).astype(np.float32)
    img = img_rgb.astype(np.float32) / 255.0
    target01 = (target_rgb.astype(np.float32) / 255.0).reshape(1, 1, 3)
    target_hsv = cv2.cvtColor(target01, cv2.COLOR_RGB2HSV).reshape(3,)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    H, S, V = hsv[..., 0], hsv[..., 1], hsv[..., 2]
    H_new = (1 - a) * H + a * ((1 - strength) * H + strength * target_hsv[0])
    S_new = (1 - a) * S + a * ((1 - strength) * S + strength * max(target_hsv[1], 0.4))
    out = cv2.cvtColor(np.stack([H_new, S_new, V], axis=-1), cv2.COLOR_HSV2RGB)
    return np.clip(out * 255, 0, 255).astype(np.uint8)

def recolor_lab_plus_hard(img_rgb: np.ndarray, mask_u8: np.ndarray, target_rgb: np.ndarray,
                          strength_ab: float = 1.0, strength_L: float = 0.3) -> np.ndarray:
    a = (mask_u8 > 0).astype(np.float32)
    img = img_rgb.astype(np.float32) / 255.0
    lab = color.rgb2lab(img)
    tgt = color.rgb2lab((target_rgb.astype(np.float32) / 255.0)[None, None, :])[0, 0, :]
    L, A, B = lab[..., 0], lab[..., 1], lab[..., 2]
    L2 = (1 - a) * L + a * ((1 - strength_L) * L + strength_L * tgt[0])
    A2 = (1 - a) * A + a * ((1 - strength_ab) * A + strength_ab * tgt[1])
    B2 = (1 - a) * B + a * ((1 - strength_ab) * B + strength_ab * tgt[2])
    out = np.clip(color.lab2rgb(np.stack([L2, A2, B2], axis=-1)), 0, 1)
    return (out * 255).astype(np.uint8)

# ---------- Parameters ----------
target_hex      = "#D32F2F"
target_rgb      = hex_to_rgb(target_hex)
strength_hsv    = 0.85
strength_lab_ab = 1.0
strength_lab_L  = 0.30

MASK_FILENAMES = {
    "BodyRaw":   "body_raw.png",
    "BodyGC":    "body_gc.png",
}

# ---------- Batch recolor over all outputs/<stem>/... ----------
stems = [d for d in os.listdir(OUT_DIR) if os.path.isdir(os.path.join(OUT_DIR, d))]
stems.sort()

processed = 0
for stem in stems:
    stem_dir = os.path.join(OUT_DIR, stem)
    if not any(os.path.isfile(os.path.join(stem_dir, fn)) for fn in MASK_FILENAMES.values()):
        continue

    img_path = find_source_image(stem)
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"Skipping '{stem}': cannot read image.")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    H, W = img_rgb.shape[:2]

    for variant_name, mask_file in MASK_FILENAMES.items():
        mask_path = os.path.join(stem_dir, mask_file)
        if not os.path.isfile(mask_path):
            continue

        try:
            mask = load_mask(mask_path, H, W)
        except FileNotFoundError as e:
            print(e)
            continue

        out_hsv = recolor_hsv_hard(img_rgb, mask, target_rgb, strength=strength_hsv)
        out_lab = recolor_lab_plus_hard(img_rgb, mask, target_rgb,
                                        strength_ab=strength_lab_ab, strength_L=strength_lab_L)

        cv2.imwrite(os.path.join(RECOLOR_DIR, f"{stem}_{variant_name}_HSV.png"),
                    cv2.cvtColor(out_hsv, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(RECOLOR_DIR, f"{stem}_{variant_name}_Lab.png"),
                    cv2.cvtColor(out_lab, cv2.COLOR_RGB2BGR))

    processed += 1

print(f"Saved recolors to: {RECOLOR_DIR}")
print(f"Processed {processed} image folder(s).")


Saved recolors to: /content/drive/MyDrive/Colab Notebooks/final_project_computer_vision/outputs/recolor
Processed 19 image folder(s).
